In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/drug-classification/drug200.csv')

**Predicted label:**

- Drug type

**Features:**

- Age of patient
- Sex of patient
- Blood Pressure Levels (BP)
- Cholesterol Levels
- Na to Potassium Ratio

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['Drug'].unique()

In [ ]:
df['Drug'] = df['Drug'].transform(lambda value: value[-1])

In [ ]:
df['Drug']

In [ ]:
df.head()

# Exploratory Data Analysis

In [ ]:
sns.countplot(x='Drug',data=df)

In [ ]:
sns.countplot(x='Drug',hue='Sex',data=df)

In [ ]:
sns.boxplot(x='Age',y='Drug',hue='Sex',data=df)

In [ ]:
sns.countplot(x='Cholesterol',data=df)

In [ ]:
sns.countplot(x='BP',data=df)

In [ ]:
sns.countplot(x='Sex',data=df)

In [ ]:
sns.jointplot(data=df,x='Age',y='Na_to_K',kind='hex')

In [ ]:
sns.catplot(data=df,x='Sex',y='Age',kind='box',
           col='Cholesterol',row='BP')

In [ ]:
sns.displot(x='Age',col='Sex',data=df,bins=15,kde=True)

In [ ]:
plt.figure(figsize=(6,4),dpi=200)
sns.scatterplot(x='Age',y='Na_to_K',hue='Drug',data=df)
plt.legend(bbox_to_anchor=(1.3,0.7),title='Drug Type')

# Feature Engineering 

In [ ]:
df['BP'].unique()

In [ ]:
df['Cholesterol'].unique()

### Ordinal Encoding

In [ ]:
bp_map = {'LOW':0,'NORMAL':1,'HIGH':2}

In [ ]:
df['BP'] = df['BP'].replace(bp_map)

In [ ]:
df.head()

 ### One-Hot Encoding

In [ ]:
dummie_df  = pd.get_dummies(df[['Cholesterol','Sex']],drop_first=True)

In [ ]:
dummie_df

In [ ]:
df = pd.concat([df,dummie_df],axis=1)

In [ ]:
df.head()

In [ ]:
df = df.drop(['Sex','Cholesterol'],axis=1)

In [ ]:
df.head()

In [ ]:
df = df[['Age','Sex_M','BP','Cholesterol_NORMAL','Na_to_K','Drug']]

In [ ]:
df.head()

In [ ]:
df.columns = ['Age','Sex_Male','BP','Cholesterol_Normal','Na_to_K','Drug']

In [ ]:
df.head()

# Random Forests Using GridSearchCV and Parameter Search

In [ ]:
X = df.drop('Drug',axis=1)

In [ ]:
y = df['Drug']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
n_estimators = [64,100,128,200]

In [ ]:
max_features = [2,3,4,5]

In [ ]:
bootstrap = [True,False]

In [ ]:
param_grid = {'n_estimators':n_estimators,
             'max_features':max_features,
             'bootstrap':bootstrap}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
grid = GridSearchCV(rfc,param_grid)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
predictions = grid.predict(X_test)

In [ ]:
predictions

In [ ]:
y_test

# Performance Evaluation

In [ ]:
from sklearn.metrics import plot_confusion_matrix,classification_report,accuracy_score

In [ ]:
plot_confusion_matrix(grid,X_test,y_test)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
errors = []
misclassifications = []

for n in range(1,200):
    rfc = RandomForestClassifier(n_estimators=n,max_features=2)
    rfc.fit(X_train,y_train)
    preds = rfc.predict(X_test)
    err = 1 - accuracy_score(y_test,preds)
    n_missed = np.sum(preds != y_test)
    
    errors.append(err)
    misclassifications.append(n_missed)

In [ ]:
plt.plot(range(1,200),errors)

In [ ]:
plt.plot(range(1,200),misclassifications)

In [ ]:
# Performance starts to level off at around 13 decision trees